<a href="https://colab.research.google.com/github/dm-fedorov/pandas_basic/blob/master/%D1%83%D0%BF%D1%80%D0%B0%D0%B6%D0%BD%D0%B5%D0%BD%D0%B8%D1%8F/1.%20%D0%A4%D1%83%D1%82%D0%B1%D0%BE%D0%BB%D0%B8%D1%81%D1%82%D1%8B.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory" target="_blank"></a>

In [ ]:
import pandas as pd
import numpy as np

Операции слияния и соединения чаще всего оказываются нужны при объединении данных из различных источников. Рассмотрим пример с определенными данными по штатам США и их населению.

Посмотрим на эти наборы данных с помощью функции read_csv() библиотеки Pandas:

In [ ]:
pop = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv')
pop.head()

In [ ]:
areas = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv')
areas.head()

In [ ]:
abbrevs = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv')
abbrevs.head()

Допустим, нам нужно на основе этой информации отсортировать штаты и территорию США по плотности населения в 2010 году. Информации для этого у нас достаточно, но для достижения цели придется объединить наборы данных.
Начнем со слияния «многие-ко-многим», которое позволит получить полное имя штата в объекте DataFrame для населения. Выполнить слияние нужно на основе столбца state/region объекта pop и столбца abbreviation объекта abbrevs . Мы воспользуемся опцией how='outer' , чтобы гарантировать, что не упустим никаких данных из-за несовпадения меток.

In [ ]:
merged = pd.merge(pop, abbrevs, how='outer', left_on='state/region', right_on='abbreviation')
merged.head()

Удаляем дублирующуюся информацию:

In [ ]:
merged = merged.drop('abbreviation', 1)
merged.head()

Следует проверить, не было ли каких-то несовпадений. Сделать это можно путем поиска строк с пустыми значениями:

In [ ]:
merged.isnull().any()

Часть информации о населении отсутствует, выясним, какая именно:

In [ ]:
merged[merged['population'].isnull()].head()

Похоже, что источник пустых значений по населению — Пуэрто-Рико, до 2000 года.
Вероятно, это произошло из-за того, что необходимых данных не было в первоисточнике.
Мы видим, что некоторые из новых значений столбца state тоже пусты, а значит, в ключе объекта abbrevs отсутствовали соответствующие записи! Выясним, для каких территорий отсутствуют соответствующие значения:

In [ ]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

Все понятно: наши данные по населению включают записи для Пуэрто-Рико (PR) и США в целом (USA), отсутствующие в ключе аббревиатур штатов. Исправим это, вставив соответствующие записи:

In [ ]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

В столбце state больше нет пустых значений. Готово!

Теперь можно слить результат с данными по площади штатов с помощью аналогичной процедуры. После изучения имеющихся результатов становится понятно, что нужно выполнить соединение по столбцу state в обоих объектах:

In [ ]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

Выполним снова проверку на пустые значения, чтобы узнать, были ли какие-то несовпадения:

In [ ]:
final.isnull().any()

В столбце area имеются пустые значения. Посмотрим, какие территории не были учтены:

In [ ]:
final['state'][final['area (sq. mi)'].isnull()].unique()

Видим, что наш DataFrame -объект areas не содержит площадь США в целом.
Мы могли бы вставить соответствующее значение (например, воспользовавшись суммой площадей всех штатов), но в данном случае мы просто удалим пустые значения, поскольку плотность населения США в целом нас сейчас не
интересует:

In [ ]:
final.dropna(inplace=True)
final.head()

Теперь у нас есть все необходимые нам данные. Чтобы ответить на интересующий вопрос, сначала выберем часть данных, соответствующих 2010 году и всему населению.

In [ ]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

Теперь вычислим плотность населения и выведем данные в соответствующем порядке. Начнем с переиндексации наших данных по штату, после чего вычислим результат:

In [ ]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']

In [ ]:
density.sort_values(ascending=False, inplace=True)
density.head()

Результат — список штатов США плюс Вашингтон (округ Колумбия) и Пуэрто-Рико, упорядоченный по плотности населения в 2010 году, в жителях на квадратную милю. Как видим, самая густонаселенная территория в этом наборе данных — Вашингтон (округ Колумбия); среди штатов же самый густонаселенный — Нью-Джерси.
Можно также вывести окончание списка:

In [ ]:
density.tail()

Как видим, штатом с наименьшей плотностью населения, причем с большим отрывом от остальных, оказалась Аляска, насчитывающая в среднем одного жителя на квадратную милю.

Подобное громоздкое слияние данных — распространенная задача при ответе на вопросы, связанные с реальными источниками данных. Надеюсь, что этот пример дал вам представление, какими способами можно комбинировать вышеописанные инструменты, чтобы почерпнуть полезную информацию из данных!